In [1]:
from pyspark.sql import SparkSession
from configparser import ConfigParser
parser = ConfigParser()
parser.read('config.ini')

spark = SparkSession.builder\
    .master("local[*]")\
    .appName("Assignmet")\
    .config('spark.driver.extraClassPath','sqljdbc41.jar')\
    .config('spark.executor.extraClassPath', 'sqljdbc41.jar')\
    .getOrCreate()

22/05/10 04:30:54 WARN Utils: Your hostname, MohammadFatha resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp1s0)
22/05/10 04:30:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/10 04:30:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.json('reviews_Electronics_5.json')
df.show(10)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0528881469|  [0, 0]|    5.0|We got this GPS f...| 06 2, 2013| AO94DHGC771SJ|             amazdnu|     Gotta have GPS!|    1370131200|
|0528881469|[12, 15]|    1.0|I'm a professiona...|11 25, 2010| AMO214LNFCEI4|     Amazon Customer|   Very Disappointed|    1290643200|
|0528881469|[43, 45]|    3.0|Well, what can I ...| 09 9, 2010|A3N7T0DY83Y4IG|       C. A. Freeman|      1st impression|    1283990400|
|0528881469| [9, 10]|    2.0|Not going to writ...|11 24, 2010|A1H8PY3QHMQQA0|Dave M. Shaw "mac...|Great grafics, PO...|    1290556800|
|0528881469|  [0, 0]|    1.0|I've had mine for...|09 29

In [3]:
df.printSchema()


root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [4]:
from pyspark.sql.functions import *
from datetime import datetime
from pyspark.sql.types import DateType

In [5]:
func =  udf (lambda x: datetime.strptime(x, '%m %d, %Y'), DateType())

df1 = df.withColumn('new_col', date_format(func(col('reviewTime')), 'MM-dd-yyy'))
df1 = df1.withColumnRenamed('new_col', 'NewreviewTime')
df1 = df1.drop('reviewTime')
df1.show(5)
df1.printSchema()

+----------+--------+-------+--------------------+--------------+--------------------+--------------------+--------------+-------------+
|      asin| helpful|overall|          reviewText|    reviewerID|        reviewerName|             summary|unixReviewTime|NewreviewTime|
+----------+--------+-------+--------------------+--------------+--------------------+--------------------+--------------+-------------+
|0528881469|  [0, 0]|    5.0|We got this GPS f...| AO94DHGC771SJ|             amazdnu|     Gotta have GPS!|    1370131200|   06-02-2013|
|0528881469|[12, 15]|    1.0|I'm a professiona...| AMO214LNFCEI4|     Amazon Customer|   Very Disappointed|    1290643200|   11-25-2010|
|0528881469|[43, 45]|    3.0|Well, what can I ...|A3N7T0DY83Y4IG|       C. A. Freeman|      1st impression|    1283990400|   09-09-2010|
|0528881469| [9, 10]|    2.0|Not going to writ...|A1H8PY3QHMQQA0|Dave M. Shaw "mac...|Great grafics, PO...|    1290556800|   11-24-2010|
|0528881469|  [0, 0]|    1.0|I've had min

In [9]:
df1 = df1.withColumn("helpful",df1['helpful'].cast('string'))
df1.printSchema()

In [14]:

database = parser.get('config-txt', 'sqldatabase')
table  = parser.get('config-txt', 'table')
user = parser.get('config-txt', 'user')
password = parser.get('config-txt', 'password')

In [10]:

df1.write.mode("overwrite").format("jdbc")\
    .option("url", f"jdbc:sqlserver://localhost:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()
 

In [11]:
df1.write.parquet('Assignmentparaquetfile')